# Notebook Import

In [1]:
import pandas as pd
import numpy as np

# Constants

In [2]:
TRAINING_DATA_FILE="SpamData/02_Training/trian-data.txt"
TEST_DATA_FILE="SpamData/02_Training/test-data.txt"

TOKEN_SPAM_PROB_FILE="SpamData/02_Training/prob-spam.txt"
TOKEN_HAM_PROB_FILE="SpamData/02_Training/prob-ham.txt"
TOKEN_ALL_PROB_FILE="SpamData/02_Training/prob-all-tokens.txt"

TEST_FEATURE_MATRIX="SpamData/03_Testing/test-features.txt"
TEST_TARGET_FILE="SpamData/03_Testing/test-target.txt"


VOCAB_SIZE=2500

# Read and Load Features from .txt file into Data Frames

In [3]:
sparse_train_data=np.loadtxt(TRAINING_DATA_FILE,delimiter=' ',dtype=int)

In [4]:
sparse_test_data=np.loadtxt(TEST_DATA_FILE,delimiter=' ',dtype=int)

In [5]:
print("Number of rows in training file",sparse_train_data.shape[0])
print("Number of rows in test file",sparse_test_data.shape[0])

Number of rows in training file 304277
Number of rows in test file 71653


In [6]:
print("Number of unique in training file",len(set(sparse_train_data[:,0])))
print("Number of unique in test file",len(set(sparse_test_data[:,0])))

Number of unique in training file 4589
Number of unique in test file 1150


## Create Empty DataFrame

In [7]:
index_names=np.unique(sparse_train_data[:,0])
index_names

array([   0,    1,    4, ..., 5793, 5794, 5795])

In [8]:
sparse_train_data

array([[   0,    2,    1,    1],
       [   0,    3,    1,    2],
       [   0,    4,    1,    1],
       ...,
       [5795, 2287,    0,    1],
       [5795, 2356,    0,    1],
       [5795, 2417,    0,    1]])

In [9]:
column_names=['DOC_ID']+['CATEGORY']+list(range(0,VOCAB_SIZE))

In [10]:
full_train_data=pd.DataFrame(index=index_names, columns=column_names)

In [11]:
full_train_data.fillna(0, inplace=True)

In [42]:
sparse_test_data.shape

(71653, 4)

# Create FULL Matrix from Sparse Matrix

In [13]:
sparse_train_data

array([[   0,    2,    1,    1],
       [   0,    3,    1,    2],
       [   0,    4,    1,    1],
       ...,
       [5795, 2287,    0,    1],
       [5795, 2356,    0,    1],
       [5795, 2417,    0,    1]])

In [14]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    
    Forms a full matrix from the Sparse MAtrix
    sparse_matrix -- numpy array
    nr_words -- Vocab size
    doc_idx -- Position of document in the matrix , deafult 1st position
    word_idx -- Position word_id in the sparse matrix , default 2nd position
    cat_idx -- Position of Label 
    """
    column_names=['DOC_ID']+['CATEGORY']+list(range(0,VOCAB_SIZE))
    doc_id_names=np.unique(sparse_matrix[:,0])
    full_matrix=pd.DataFrame(index=index_names, columns=column_names)
    full_matrix.fillna(0, inplace=True)
    for i in range(sparse_matrix.shape[0]):
        DOC_ID=sparse_matrix[i][doc_idx]
        CATEGORY=sparse_matrix[i][cat_idx]
        word_id = sparse_matrix[i][word_idx]
        occ=sparse_matrix[i][freq_idx]
        full_matrix.at[DOC_ID,"DOC_ID"]=DOC_ID
        full_matrix.at[DOC_ID,"CATEGORY"]=CATEGORY
        full_matrix.at[DOC_ID,word_id]=occ
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix

In [15]:
%%time
full_train_data=make_full_matrix(sparse_train_data,VOCAB_SIZE)

Wall time: 11.9 s


In [16]:
full_train_data

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,1,3,2,1,2,2,1,...,0,0,0,0,0,0,0,0,0,0
7,1,1,1,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5787,0,2,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5793,0,1,9,1,0,0,1,2,1,1,...,0,0,0,0,0,0,0,0,0,0


## Calculate Probability of Spam

In [17]:
spam_size=full_train_data[full_train_data.CATEGORY==1].shape[0]
prob_size=spam_size/full_train_data.shape[0]

In [18]:
prob_size

0.3216387012421007

## Total Number Of Tokens

In [19]:
full_train_features=full_train_data.loc[:,full_train_data.columns != 'CATEGORY']

In [20]:
email_length=full_train_features.sum(axis=1)

In [21]:
email_length

DOC_ID
0        87
1        53
4        43
5       141
7       111
       ... 
5787     38
5790     72
5793    227
5794     57
5795    264
Length: 4589, dtype: int64

In [22]:
total_wc=email_length.sum()

## Number of Tokens in Spams & Ham Emails

In [23]:
spam_lengths=email_length[full_train_data.CATEGORY==1]
#spam_lengths=spam_data.sum(axis=1)
tot_spam_wc=spam_lengths.sum()

In [24]:
ham_lengths=email_length[full_train_data.CATEGORY==0]
#ham_lengths=ham_data.sum(axis=1)
tot_ham_wc=ham_lengths.sum()

In [25]:
print("Average nr of words in spam mails {:.0f}".format(tot_spam_wc/spam_lengths.shape[0]))
print("Average nr of words in ham mails {:.3f}".format(tot_ham_wc/ham_lengths.shape[0]))

Average nr of words in spam mails 151
Average nr of words in ham mails 94.231


## Sum tokens Occuring in Spam

In [26]:
train_spam_tokens=full_train_features.loc[full_train_data.CATEGORY==1]
train_ham_tokens=full_train_features.loc[full_train_data.CATEGORY==0]

In [27]:
train_spam_tokens

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,1,3,2,1,2,2,1,1,...,0,0,0,0,0,0,0,0,0,0
7,1,1,0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1890,2,0,0,0,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1892,2,0,2,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1893,7,0,3,2,2,2,1,3,1,1,...,0,0,0,0,0,0,0,0,0,0


In [28]:
summed_spam_tokens=train_spam_tokens.sum()+1 ## Laplace Smoothing
summed_ham_tokens=train_ham_tokens.sum()+1

In [29]:
summed_spam_tokens

0       2330
1       1133
2       1591
3       2441
4       1490
        ... 
2495      45
2496      22
2497      23
2498      18
2499      25
Length: 2500, dtype: int64

## P(Token|Spam) - Probability that token occurs given Email is Spam

In [30]:
prob_tokens_spam=summed_spam_tokens/(tot_spam_wc+VOCAB_SIZE)
prob_tokens_spam.sum()

1.0

In [31]:
prob_tokens_ham=summed_ham_tokens/(tot_ham_wc+VOCAB_SIZE)
prob_tokens_ham.sum()

1.0000000000000002

# P(Token) - Probability that Token Occurs

In [32]:
prob_tokens_all=full_train_features.sum(axis=0)/total_wc

In [33]:
prob_tokens_all.sum()

1.0

## Save the Trained Model

In [34]:
np.savetxt(TOKEN_SPAM_PROB_FILE,prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE,prob_tokens_ham)
np.savetxt(TOKEN_ALL_PROB_FILE,prob_tokens_all)

## Prepare Test Data

In [43]:
%%time
index_names=np.unique(sparse_test_data[:,0])
test_data=make_full_matrix(sparse_test_data,VOCAB_SIZE)

Wall time: 4.41 s


In [44]:
test_data

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,13,0,...,0,0,0,0,0,0,0,0,0,0
6,1,2,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
10,1,3,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
13,1,1,0,0,1,0,1,0,2,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5781,0,2,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
5788,0,0,4,0,2,4,3,3,1,4,...,0,0,0,0,0,0,0,0,0,0
5789,0,3,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
y_test=test_data["CATEGORY"]
X_test=test_data.loc[:,test_data.columns !="CATEGORY"]

In [46]:
np.savetxt(TEST_FEATURE_MATRIX,X_test)
np.savetxt(TEST_TARGET_FILE,y_test)

In [47]:
sparse_test_data.shape

(71653, 4)

In [48]:
sparse_train_data.shape

(304277, 4)